<a href="https://colab.research.google.com/github/be-next/TP-IA/blob/main/TP_IA_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.__version__

'1.5.3'

## Chargement des données dans dataframe Pandas  
  
  - Dataset d'origine : https://archive.ics.uci.edu/dataset/50/image+segmentation
  - Copie sur GitHub : https://raw.githubusercontent.com/be-next/TP-IA/main/datasets/image_segmentation.csv


In [ ]:
url = "https://raw.githubusercontent.com/be-next/TP-IA/main/datasets/segmentation_test.csv"
D = pd.read_csv(url)

# D.describe()
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CLASS                 2100 non-null   object 
 1   REGION-CENTROID-COL   2100 non-null   float64
 2   REGION-CENTROID-ROW   2100 non-null   float64
 3   REGION-PIXEL-COUNT    2100 non-null   int64  
 4   SHORT-LINE-DENSITY-5  2100 non-null   float64
 5   SHORT-LINE-DENSITY-2  2100 non-null   float64
 6   VEDGE-MEAN            2100 non-null   float64
 7   VEDGE-SD              2100 non-null   float64
 8   HEDGE-MEAN            2100 non-null   float64
 9   HEDGE-SD              2100 non-null   float64
 10  INTENSITY-MEAN        2100 non-null   float64
 11  RAWRED-MEAN           2100 non-null   float64
 12  RAWBLUE-MEAN          2100 non-null   float64
 13  RAWGREEN-MEAN         2100 non-null   float64
 14  EXRED-MEAN            2100 non-null   float64
 15  EXBLUE-MEAN          

In [ ]:
D['CLASS'].value_counts()

GRASS        300
PATH         300
WINDOW       300
CEMENT       300
FOLIAGE      300
SKY          300
BRICKFACE    300
Name: CLASS, dtype: int64

In [ ]:
X = D[D.columns[1:]]
# X.info()
X.shape

(2100, 19)

In [ ]:
y = D['CLASS']
y.shape

(2100,)

In [ ]:
import sklearn as sk
sk.__version__

'1.2.2'

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1310, random_state=12345)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1310, 19)
(790, 19)
(1310,)
(790,)


convertion des valeurs catégorielles de y_train en un format de données "one-hot encoding"

In [ ]:
y_train_encoded = pd.get_dummies(y_train)
y_train_encoded.head()

,BRICKFACE,CEMENT,FOLIAGE,GRASS,PATH,SKY,WINDOW
160,0,0,0,0,0,0,1
1833,1,0,0,0,0,0,0
330,0,1,0,0,0,0,0
249,0,1,0,0,0,0,0
1437,0,1,0,0,0,0,0


In [ ]:
import numpy as np
np.unique(y_train, return_counts=True) #return_counts=True renvoie un tableau qui contient le nombre de fois que chaque valeur unique apparaît dans y_train


(array(['BRICKFACE', 'CEMENT', 'FOLIAGE', 'GRASS', 'PATH', 'SKY', 'WINDOW'],
       dtype=object),
 array([190, 197, 186, 184, 185, 188, 180]))

In [ ]:
np.sum(y_train_encoded.values, axis=0)

array([190, 197, 186, 184, 185, 188, 180], dtype=uint64)

### Normalisation des features

Normalisation avec `StandardScaler()`.

Après la transformation, chaque feature aura une moyenne approximativement égale à 0 et une variance égale à 1, à condition que les caractéristiques originales suivent une distribution normale. Si ce n'est pas le cas, la standardisation peut ne pas donner une distribution normale.

La transformation est :

$$
z = \frac{x - \mu}{\sigma}
$$

où

  - $z$ est la valeur transformée de la caractéristique (feature).
  - $x$ est la valeur originale de la caractéristique.
  - $μ$ est la moyenne de la caractéristique calculée sur l'ensemble des données.
  - $σ$ est l'écart-type de la caractéristique.


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

np.mean(X_train_scaled, axis=0)

array([-7.59358649e-17, -2.71199517e-18,  0.00000000e+00, -9.22078359e-17,
        3.25439421e-17,  8.13598552e-18, -1.08479807e-17, -1.62719710e-17,
       -2.71199517e-18, -8.27158528e-17, -8.27158528e-17, -4.74599156e-17,
        2.44079566e-17,  9.35638335e-17, -1.28141772e-16, -1.35599759e-18,
        1.12547800e-16,  1.23395780e-16,  7.96648583e-17])

In [ ]:
X_test_scaled = scaler.transform(X_test)
np.mean(X_test_scaled, axis=0)

array([-0.02020605,  0.01884811,  0.        , -0.04424365,  0.00630796,
       -0.03388503, -0.02926025,  0.03663427, -0.0278119 , -0.02480634,
       -0.02411881, -0.0276192 , -0.02182064,  0.02643913, -0.03907171,
        0.03982831, -0.02754092,  0.03332159,  0.00656877])

## Construction du modèle

### RN simple couche

In [ ]:
import keras
keras.__version__

'2.14.0'

In [ ]:
from keras.models import Sequential # RN avec des couches en séquence
from keras.layers import Dense # Les couches sont pleinement connectées

In [ ]:
model = Sequential()

# ajout de la couche "entrée (19) -> sortie (7)"
# y_train_encoded.shape[1] : nb de neuronnes en sortie, 1 neuronne par classe (one hot encoding)
# X_train_scaled.shape[1] : nb de neuronnes en entrée, 1 neuronne par valeur explicative

model.add( Dense(units=y_train_encoded.shape[1], input_dim=X_train_scaled.shape[1], activation='softmax' ))

model.get_config()


#### Fonction de perte `categorical_crossentropy`

Classification Multi-classes : Dans un problème de classification multi-classes, chaque échantillon appartient à une et une seule parmi plusieurs classes. Par exemple, une image pourrait être classée comme un chat, un chien, un oiseau, etc., mais pas plusieurs à la fois.

Codage One-Hot : Chaque étiquette de classe est convertie en un vecteur "one-hot". Dans ce vecteur, un seul élément est "1" (représentant la classe à laquelle l'échantillon appartient), et tous les autres éléments sont "0". Par exemple, si nous avons trois classes (chat, chien, oiseau), et que l'échantillon est un "chien", il pourrait être représenté comme [0, 1, 0].

La `categorical_crossentropy` mesure la distance entre deux distributions de probabilité - la distribution de probabilité prédite par le modèle et la distribution de probabilité réelle (les étiquettes one-hot).

Sa formule est :

$$
-\sum_{c=1}^{M} y_{\text{true}, c} \log(y_{\text{pred}, c})
$$


où
  - $M$ est le nombre total de classes,
  - $y_{\text{true}, c}$ est la valeur binaire de la classe réelle pour la classe $c$ dans le vecteur one-hot,
  - $y_{\text{pred}, c}$ est la probabilité prédite par le modèle pour la classe $c$.

Interprétation :

Si la prédiction du modèle est parfaitement exacte (la probabilité prédite pour la classe correcte est 1), la perte est 0 car $\log(1)=0$.
Plus la prédiction s'éloigne de la réalité (la probabilité pour la classe correcte diminue), plus la perte augmente.

Objectif en Apprentissage :

Durant l'entraînement, l'objectif est de minimiser cette fonction de perte. Une faible valeur de "categorical_crossentropy" indique que le modèle est bon pour prédire la classe correcte avec une forte probabilité.

#### Optimiseur Adam (Adaptive Moment Estimation)

Principes de Base

Combinaison des Avantages de Deux Autres Optimiseurs : Adam combine les idées de deux autres méthodes d'optimisation populaires : RMSprop et Momentum. Comme Momentum, Adam conserve une moyenne mobile (décroissante exponentiellement) des gradients passés (momentum), et comme RMSprop, il conserve une moyenne mobile des carrés des gradients passés.

Mise à Jour des Poids : L'objectif de l'optimiseur Adam est de mettre à jour les poids du réseau de manière efficace en ajustant les taux d'apprentissage de chaque poids, ce qui peut aider à accélérer l'apprentissage et à améliorer la convergence.



In [ ]:
# configuration pour l'apprentissage
# Nous somme dans la résolution d'un problème de classement
# Choix de l'algorithme de descente de gradien stokastique
# la métrique nous donne le taux de reconnaissance

model.compile( loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



Entrainement sur l'échantillon de données d'apprentissage

In [ ]:
model.fit(X_train_scaled, y_train_encoded, epochs=150, batch_size=20)

Évalusation de la pertinence du model sur l'échantillon de test

In [ ]:
y_predict_encoded = model.predict(X_test_scaled)
y_predict_encoded.shape

25/25 [==============================] - 0s 3ms/step


(790, 7)

In [ ]:
y_predict_encoded[:5]

array([[6.7642318e-08, 1.1638989e-02, 2.3485827e-08, 2.3899808e-08,
        5.4523145e-04, 9.8781550e-01, 2.2782802e-07],
       [1.3870419e-02, 6.7386365e-01, 3.5513133e-02, 1.6133652e-04,
        1.0111734e-04, 1.1860055e-05, 2.7647844e-01],
       [1.9374905e-02, 1.4679595e-04, 8.4539759e-01, 6.3536500e-05,
        3.9093072e-07, 7.7649418e-08, 1.3501668e-01],
       [1.4497743e-11, 3.8526316e-05, 6.8556684e-13, 1.8977642e-10,
        2.0429272e-07, 9.9996120e-01, 4.1308870e-12],
       [2.0105533e-08, 1.0231293e-02, 9.8888153e-01, 5.2559708e-06,
        8.9183311e-07, 2.4517565e-08, 8.8091026e-04]], dtype=float32)

In [ ]:
np.sum(y_predict_encoded, axis=1)[:5]

array([1.        , 0.9999999 , 1.        , 0.99999994, 0.9999999 ],
      dtype=float32)

In [ ]:
y_predict_decoded = np.argmax(y_predict_encoded, axis=1)
y_predict_decoded[:5]

array([5, 1, 2, 5, 2])

In [ ]:
y_train_encoded.columns

Index(['BRICKFACE', 'CEMENT', 'FOLIAGE', 'GRASS', 'PATH', 'SKY', 'WINDOW'], dtype='object')

In [ ]:
y_predict = y_train_encoded.columns[y_predict_decoded]
y_predict[:5]

Index(['SKY', 'CEMENT', 'FOLIAGE', 'SKY', 'FOLIAGE'], dtype='object')

Matrice de confusion

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_predict)

array([[109,   0,   0,   0,   0,   0,   1],
       [  1,  92,   2,   0,   2,   0,   6],
       [  1,   1,  99,   0,   0,   0,  13],
       [  0,   0,   0, 115,   1,   0,   0],
       [  0,   0,   0,   0, 115,   0,   0],
       [  0,   0,   0,   0,   0, 112,   0],
       [  0,  11,  23,   0,   0,   0,  86]])

Accuracy

## Résultats

Globalement c'est bien, l'accuracy est de 92%.
Mais dans le détail, on voit que la classe 'WINDOW' n'est pas très bien reconnue.
Il y a également un problème sur la classe 'FOLIAGE'

In [ ]:
metrics.accuracy_score(y_test, y_predict)

0.9215189873417722

In [ ]:
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

   BRICKFACE       0.98      0.99      0.99       110
      CEMENT       0.88      0.89      0.89       103
     FOLIAGE       0.80      0.87      0.83       114
       GRASS       1.00      0.99      1.00       116
        PATH       0.97      1.00      0.99       115
         SKY       1.00      1.00      1.00       112
      WINDOW       0.81      0.72      0.76       120

    accuracy                           0.92       790
   macro avg       0.92      0.92      0.92       790
weighted avg       0.92      0.92      0.92       790



## Évaluation avec Keras

In [ ]:
model.evaluate(X_test_scaled, pd.get_dummies(y_test))

25/25 [==============================] - 0s 1ms/step - loss: 0.2130 - accuracy: 0.9215


[0.2130223959684372, 0.9215189814567566]

On retrouve le calcul d'accuracy calculé avec SciKit Learn ...
Ce sont les mêmes travaux d'évaluation exécutés avec 2 librairies différentes (ouff ;) ).

# RN multi-couche

In [ ]:
model_mc = Sequential()

hiden_layer_size = 10

# Ajout de la première couche "entrée (19) -> couche cachée (10)"
model_mc.add( Dense(units=hiden_layer_size, input_dim=X_train_scaled.shape[1], activation='relu' ))

# Ajout de la seconde couche "couche cachée (10) -> couche de sortie (7)"
# Pas d'input_dim à spécifier
model_mc.add( Dense( units=y_train_encoded.shape[1], activation='softmax'))

model_mc.get_config()

In [ ]:
# configuration pour l'apprentissage
# Nous somme dans la résolution d'un problème de classement
# Choix de l'algorithme de descente de gradien stokastique
# la métrique nous donne le taux de reconnaissance

model_mc.compile( loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model_mc.fit(X_train_scaled, y_train_encoded, epochs=150, batch_size=20)

In [ ]:
model_mc.evaluate(X_test_scaled, pd.get_dummies(y_test))